<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Use-case-1:-Identify-people-wearing-PPE-(Personal-Protective-Equipment)-with-YoloV5" data-toc-modified-id="Use-case-1:-Identify-people-wearing-PPE-(Personal-Protective-Equipment)-with-YoloV5-1">Use case 1: Identify people wearing PPE (Personal Protective Equipment) with YoloV5</a></span><ul class="toc-item"><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.1">Dataset</a></span></li><li><span><a href="#Testing-the-model-with-Videos" data-toc-modified-id="Testing-the-model-with-Videos-1.2">Testing the model with Videos</a></span></li></ul></li><li><span><a href="#Use-case-2:-Pose-Detection-with-OpenPose" data-toc-modified-id="Use-case-2:-Pose-Detection-with-OpenPose-2">Use case 2: Pose Detection with OpenPose</a></span></li></ul></div>

# Use case 1: Identify people wearing PPE (Personal Protective Equipment) with YoloV5



In [1]:
# We first need to clone YoloV5 into our Colab notebook

!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [2]:
# This is the Hardware Google is providing us with
!nvidia-smi

Tue Feb 23 10:31:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    11W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Dataset

**First batch**
1.   Download the dataset (https://drive.google.com/drive/folders/1M8nzvcnAsEXwz81x18X_mGeqPztZBIvO)
2.   Labeling: 60 labels each team member = total 360 images labeled:
  *   W = Worker
  *   WH = Worker + Helmet
  *   WV = Worker + Vest
  *   WHV = Worker + Helmet + Vest
3.   We uploaded the dataset + labels to Roboflow (https://app.roboflow.com/ds/1Ru0OwInh8?key=4SXkhvp9Bz)
4.   We split the dataset into 80% training and 20% validation
5.   We used a YAML file to tell YoloV5 model how to process the images


**Second batch**

We realize, we needed to increase the amount of pictures of the class WHV
https://www.kaggle.com/andrewmvd/hard-hat-detection

For that 1gb dataset, we select 60 images of workers with vests. After that we used an online service to transform them from *.png to *.jpg (https://png2jpg.com). 

Then, we used again LabelMe to label the images, and finally we uploaded them into RoboFlow to update the previous dataset.

**Final Roboflow file** with all the images (1st and 2nd batches):https://app.roboflow.com/ds/li8qFcJ6Sn?key=xBr8ACleFG

In [3]:
# Extracting the dataset we uploaded to Roboflow
%cd /content
!curl -L "https://app.roboflow.com/ds/li8qFcJ6Sn?key=xBr8ACleFG" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   1244      0 --:--:-- --:--:-- --:--:--  1242
100 28.5M  100 28.5M    0     0  23.2M      0  0:00:01  0:00:01 --:--:-- 23.2M
Archive:  roboflow.zip
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: train/
   creating: train/images/
 extracting: train/images/hard_hat_workers1003.rf.f8b8a4e142f2fb616b012f429a600747.jpg  
 extracting: train/images/hard_hat_workers1048.rf.faf433d3060728f840c2087bb9021831.jpg  
 extracting: train/images/hard_hat_workers1048_jpg.rf.cb0e2b4b6ec378a1b786e93f9b0d94b7.jpg  
 extracting: train/images/hard_hat_workers1053.rf.00aeb2dd7233176e2754bc9930e9ab63.jpg  
 extracting: train/images/hard_hat_workers1053_jpg.rf.d6ca14a03653fe3de5b1a4e4299c7ab3.jpg  
 extracting: train/images/hard_hat_workers1086.rf.b724305bfc069067c5f9e12e46

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
# YAML (a recursive acronym for "Yet Ain't Markup Language") is a human-readable data-serialization language. 
# It is commonly used for configuration files and in applications where data is being stored or transmitted
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 4
names: ['W', 'WH', 'WHV', 'WV']

In [4]:
# Define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [6]:
# This is the model configuration we will use as base for our use case
%cat /content/yolov5/models/yolov5s.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C

In [7]:
# Customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [8]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# We will train the YoloV5s on our PPE dataset and time it
%%time 
%cd /content/yolov5/
!python train.py --img 160 --batch 32 --epochs 1000 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 v4.0-95-gcc79f3a torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=32, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='../data.yaml', device='', epochs=1000, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[160, 160], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='yolov5s_results', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5s_results', single_cls=False, sync_bn=False, total_batch_size=32, weights='', workers=8, world_size=1)
wandb: Install Weights & Biases for YOLOv5 logging with 'pip install wandb' (recommended)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-02-23 10:32:47.823001: I tensorflow/stream_executor

In [ ]:
# Start tensorboard --> launch after you have started training
# logs are saved in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

train/


In [ ]:
%ls runs/train/yolov5s_results/weights

best.pt  last.pt


In [ ]:
# when we ran this, we saw 0.013 second inference time. That is 140 FPS on a TESLA T4!
# use the best weights!
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.7 --source ../valid/images

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.7, device='', exist_ok=False, img_size=416, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='../valid/images', update=False, view_img=False, weights=['runs/train/yolov5s_results/weights/best.pt'])
YOLOv5 v4.0-93-g95aefea torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients, 16.8 GFLOPS
image 1/145 /content/yolov5/../valid/images/image_from_china-1015-_jpg.rf.9af7b2b368950bb7102c3ffef5989931.jpg: 416x416 2 WHs, Done. (0.011s)
image 2/145 /content/yolov5/../valid/images/image_from_china-1017-.rf.19b32f0e02357bd8a57aba1bc4d93dd8.jpg: 416x416 1 WH, Done. (0.009s)
image 3/145 /content/yolov5/../valid/images/image_from_china-1067-.rf.3a44e2da17fed04bf8b5f7d0b9eaccd6.jpg: 416x416 1 WH, Done. (0.009s)
image 4/145 /content/yolov5/../valid/images/image_from_china-1133-_jpg.rf.fef5eb65497a3605112

In [ ]:
# Display inference on ALL valid images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# We are mounting a drive, with our account

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# We are copying the "best weights" into our drive --> not necessary step, only if we want to save the weights

%cp /content/yolov5/runs/train/yolov5s_results/weights/best.pt /content/gdrive/MyDrive/ 

## Testing the model with Videos

We will try the model on different construction videos: 

1.   Fast construction: https://www.youtube.com/watch?v=N6f_sayw0mM&ab_channel=WallStreetJournal
2.   Workers dance: https://www.youtube.com/watch?v=GEC2t5L7KKQ&ab_channel=JunranXu
3.   Pouring Concrete: https://www.youtube.com/watch?v=oHEWcefPWws&ab_channel=Local528CementMasons%26Plasterers

We downloaded those videos from youtube and uploaded them to our Google Drive

In [ ]:
# 1. Fast construction: 
# Here we will use our model with the "construction_video.mp4". For that we need to upload the video into gdrive folder: "yolov5"
# To upload the video: right click on "/content/yolov5" --> "upload" --> select the video from our Teams folders "construction_video.mp4"

!python detect.py --source construction_video.mp4 --weights /content/yolov5/runs/train/yolov5s_results/weights/best.pt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='construction_video.mp4', update=False, view_img=False, weights=['/content/yolov5/runs/train/yolov5s_results/weights/best.pt'])
YOLOv5 v4.0-93-g95aefea torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients, 16.8 GFLOPS
video 1/1 (1/3023) /content/yolov5/construction_video.mp4: 384x640 Done. (0.010s)
video 1/1 (2/3023) /content/yolov5/construction_video.mp4: 384x640 Done. (0.009s)
video 1/1 (3/3023) /content/yolov5/construction_video.mp4: 384x640 Done. (0.009s)
video 1/1 (4/3023) /content/yolov5/construction_video.mp4: 384x640 Done. (0.009s)
video 1/1 (5/3023) /content/yolov5/construction_video.mp4: 384x640 Done. (0.009s)
video 1/1 (6/3023) /content/yolov5/construction_video.mp4: 384x640 Done. (0.009s)
video 1/1

In [ ]:
# 2. Workers dance: 

!python detect.py --source construction_video_dance.mp4 --weights /content/yolov5/runs/train/yolov5s_results/weights/best.pt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='construction_video_dance.mp4', update=False, view_img=False, weights=['/content/yolov5/runs/train/yolov5s_results/weights/best.pt'])
YOLOv5 v4.0-93-g95aefea torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients, 16.8 GFLOPS
video 1/1 (1/1184) /content/yolov5/construction_video_dance.mp4: 640x384 2 Ws, 5 WHs, Done. (0.011s)
video 1/1 (2/1184) /content/yolov5/construction_video_dance.mp4: 640x384 2 Ws, 5 WHs, Done. (0.010s)
video 1/1 (3/1184) /content/yolov5/construction_video_dance.mp4: 640x384 2 Ws, 4 WHs, Done. (0.010s)
video 1/1 (4/1184) /content/yolov5/construction_video_dance.mp4: 640x384 2 Ws, 5 WHs, Done. (0.010s)
video 1/1 (5/1184) /content/yolov5/construction_video_dance.mp4: 640x384 2 Ws, 4 WHs, Done.

In [ ]:
# 3. Concrete Pouring: 

!python detect.py --source concrete_pour.mp4 --weights /content/yolov5/runs/train/yolov5s_results/weights/best.pt

Once the model finishes to detect on both videos, it will save the results into: Results saved to runs/detect/ ---> so we can go there, and download the videos with the model running on them



---



---



# Use case 2: Pose Detection with OpenPose

We will now develop a second model for "pose detection" which we will try on our dataset of workers images

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

In [ ]:
# 

In [ ]:
# detect poses
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4